In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [17]:
from src import SeedZoneController

import datetime as dt

# Make New Cluster

In [47]:
sz = SeedZoneController()

sz.make_new_cluster()
sz.save()

ECV 53%
ECV 57%
ECV 58%
ECV 58%
ECV 58%
ECV 58%
ECV 58%
ECV 58%
ECV 58%
sorting okay :)
Save Okay :)
